In [63]:
### MOFA: Maxim's Once For All ###
### This version is using the Kernel Transition Matrix by default. ###

In [59]:
import copy
import random
import os
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt

from datasets.cifar100 import cifar100_get_datasets


# <ins>Classes</ins>

In [3]:
class Clamp(nn.Module):
    """
    Post-Activation Clamping Module
    Clamp the output to the given range (typically, [-128, +127])
    """
    def __init__(self, min_val=None, max_val=None):
        super().__init__()
        self.min_val = min_val
        self.max_val = max_val

    def forward(self, x):  # pylint: disable=arguments-differ
        """Forward prop"""
        return x.clamp(min=self.min_val, max=self.max_val)

In [4]:
class MOFAnet(nn.Module):
    # Maxim OFA Net
    def __init__(self, param_dict):
        super(MOFAnet, self).__init__()
        self.param_dict = param_dict
        self.in_ch = param_dict['in_ch']
        self.out_class = param_dict['out_class']
        self.n_units = param_dict['n_units']
        self.width_list = param_dict['width_list']
        self.kernel_list = param_dict['kernel_list']
        self.bias_list = param_dict['bias_list']
        self.bn = param_dict['bn']
        self.last_width = self.in_ch
        self.units = nn.ModuleList([])
        for i in range(n_units):
            self.units.append(Unit(len(self.kernel_list[i]), 
                                   self.kernel_list[i],
                                   self.width_list[i], 
                                   self.last_width, 
                                   self.bias_list[i],
                                   self.bn))
            self.last_width = self.width_list[i][-1]
        self.flatten = nn.Flatten()
        self.max_pool = nn.MaxPool2d(kernel_size=2)

        self.classifier = nn.Linear(1024, self.out_class) 
    def forward(self, x):
        for i, unit in enumerate(self.units[:-1]):
            x = unit(x)
            x = self.max_pool(x)
        x = self.units[-1](x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [5]:
class Unit(nn.Module):
    def __init__(self, depth, kernel_list, 
                 width_list, init_width, bias_list, bn=True):
        super(Unit, self).__init__()
        self.depth = depth
        self.kernel_list = kernel_list
        self.width_list = width_list
        self.bias_list = bias_list
        self.bn = bn
        self._width_list = [init_width] + width_list
        self.layers = nn.ModuleList([])
        for i in range(depth):
            self.layers.append(
                FusedConv2dReLU(self._width_list[i],
                                self._width_list[i+1],
                                self.kernel_list[i],
                                self.bias_list[i],
                                self.bn))
    def forward(self, x):
        for i in range(self.depth):
            x = self.layers[i](x)
        return x

In [6]:
class FusedConv2dReLU(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            bias=True,
            bn=True):
        super(FusedConv2dReLU, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
            
        ktm_core = torch.zeros((9, 1))
        ktm_core[4] = 1
        self.ktm = nn.Parameter(data=ktm_core, requires_grad=True)
        
        if kernel_size == 1:
            self.pad = 0
        elif kernel_size == 3:
            self.pad = 1
        else:
            raise ValueError
        self.func = F.conv2d
        self.conv2d = nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, stride=1,
                                padding=1, bias=bias)
        self.bn = bn
        if self.bn:
            self.batchnorm = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU()
        self.clamp = Clamp(min_val=-1, max_val=1)
    def forward(self, x):        
        weight = self.conv2d.weight
        bias = self.conv2d.bias
        if self.kernel_size == 1:
            flattened_weight = weight.view(weight.size(0), weight.size(1), -1, 9)
            weight = flattened_weight.to(device) @ self.ktm.to(device)
                    
        x = self.func(x, weight, bias, self.conv2d.stride, self.pad)
        if self.bn:
            x = self.batchnorm(x)
        x = self.activation(x)
#         x = self.clamp(x)
        return x

# <ins>Functions</ins>

## Batchnorm Related 

In [9]:
def make_bn_stats_false(model):
    for u_ind, unit in enumerate(model.units):
        for l_ind, layer in enumerate(unit.layers):
            model.units[u_ind].layers[l_ind].batchnorm.track_running_stats = False

    return model

def make_bn_stats_true(model):
    for u_ind, unit in enumerate(model.units):
        for l_ind, layer in enumerate(unit.layers):
            model.units[u_ind].layers[l_ind].batchnorm.track_running_stats = True

    return model

def fuse_bn(conv, bn):
    w = conv.weight
    mean = bn.running_mean
    var_sqrt = torch.sqrt(bn.running_var + bn.eps)
    beta = bn.weight
    gamma = bn.bias
    if conv.bias is not None:
        b = conv.bias
    else:
        b = mean.new_zeros(mean.shape)
    w = w * (beta / var_sqrt).reshape([conv.out_channels, 1, 1, 1])
    b = (b - mean) / var_sqrt * beta + gamma
    fused_conv = nn.Conv2d(conv.in_channels,
                         conv.out_channels,
                         conv.kernel_size,
                         conv.stride,
                         conv.padding,
                         bias=True)
    fused_conv.weight = nn.Parameter(w)
    fused_conv.bias = nn.Parameter(b)
    return fused_conv


def fuse_bn_mofa(mofa_net):
    param_dict = copy.deepcopy(mofa_net.param_dict)
    param_dict['bn'] = False
    fused_model = MOFAnet(param_dict)
    with torch.no_grad():
        fused_model.classifier.weight.copy_(mofa_net.classifier.weight)
        fused_model.classifier.bias.copy_(mofa_net.classifier.bias)
    for u_ind, unit in enumerate(mofa_net.units):
        for l_ind, layer in enumerate(unit.layers):
            fused_conv = fuse_bn(layer.conv2d, layer.batchnorm)
            fused_conv = fused_conv.to(device)
            with torch.no_grad():
                fused_model.units[u_ind].layers[l_ind].conv2d.weight.copy_(fused_conv.weight)
                fused_model.units[u_ind].layers[l_ind].conv2d.bias.copy_(fused_conv.bias)
    return fused_model

## Elastic Kernel - Depth -Width 

In [ ]:
def sample_subnet_kernel(mofa):
    param_dict = copy.deepcopy(mofa.param_dict)
    for u_ind, unit in enumerate(mofa.units):
        for l_ind, layer in enumerate(unit.layers):
            param_dict['kernel_list'][u_ind][l_ind] = random.choice([1, 3])
#     param_dict['kernel_list'][0][0] = random.choice([1, 3])
#     param_dict['kernel_list'][0][1] = random.choice([1, 3])
#     param_dict['kernel_list'][0][2] = random.choice([1, 3])
    param_dict['bn'] = False
    subnet = MOFAnet(param_dict)
    with torch.no_grad():
        subnet.classifier.weight.copy_(mofa.classifier.weight)
        subnet.classifier.bias.copy_(mofa.classifier.bias)
        for u_ind, unit in enumerate(mofa.units):
            for l_ind, layer in enumerate(unit.layers):
                subnet.units[u_ind].layers[l_ind].conv2d.weight.copy_(mofa.units[u_ind].layers[l_ind].conv2d.weight)
                subnet.units[u_ind].layers[l_ind].ktm.copy_(mofa.units[u_ind].layers[l_ind].ktm)
                if mofa.bias_list[u_ind][l_ind] is True:
                    subnet.units[u_ind].layers[l_ind].conv2d.bias.copy_(mofa.units[u_ind].layers[l_ind].conv2d.bias)
    return subnet

def update_mofa_from_subnet_kernel(mofa, subnet):
    with torch.no_grad():
        mofa.classifier.weight.copy_(subnet.classifier.weight)
        mofa.classifier.bias.copy_(subnet.classifier.bias)
        for u_ind, unit in enumerate(mofa.units):
            for l_ind, layer in enumerate(unit.layers):
                mofa.units[u_ind].layers[l_ind].conv2d.weight.copy_(subnet.units[u_ind].layers[l_ind].conv2d.weight)
                mofa.units[u_ind].layers[l_ind].ktm.copy_(subnet.units[u_ind].layers[l_ind].ktm)
                if mofa.bias_list[u_ind][l_ind] is True:
                    mofa.units[u_ind].layers[l_ind].conv2d.bias.copy_(subnet.units[u_ind].layers[l_ind].conv2d.bias)
    return mofa


def sample_subnet_depth(mofa, sample_kernel=True):
    param_dict = copy.deepcopy(mofa.param_dict)
    depth_list = []
    for u_ind in range(len(param_dict['width_list'])):
        max_depth = len(param_dict['width_list'][u_ind])
        min_depth = 1
        depth_list.append(random.randint(min_depth, max_depth))
    
    if sample_kernel:
        subnet = sample_subnet_kernel(mofa) # This is confirmed by Ji
    else:
        subnet = copy.deepcopy(mofa)
    
    param_dict = copy.deepcopy(subnet.param_dict)
    param_dict['bn'] = False
    param_dict['width_list'] = [lst[:depth_list[ind]] for ind, lst in enumerate(subnet.param_dict['width_list'])]
    param_dict['kernel_list'] = [lst[:depth_list[ind]] for ind, lst in enumerate(subnet.param_dict['kernel_list'])]
    param_dict['bias_list'] = [lst[:depth_list[ind]] for ind, lst in enumerate(subnet.param_dict['bias_list'])]
    
    subnet2 = MOFAnet(param_dict)
    
    with torch.no_grad():
        subnet2.classifier.weight.copy_(subnet.classifier.weight)
        subnet2.classifier.bias.copy_(subnet.classifier.bias)
        for u_ind, unit in enumerate(subnet2.units):
            for l_ind, layer in enumerate(unit.layers):
                subnet2.units[u_ind].layers[l_ind].conv2d.weight.copy_(subnet.units[u_ind].layers[l_ind].conv2d.weight)
                subnet2.units[u_ind].layers[l_ind].conv2d.bias.copy_(subnet.units[u_ind].layers[l_ind].conv2d.bias)
        
    return subnet2, param_dict, depth_list

def update_mofa_from_subnet_depth(mofa, subnet):
    subnet_params = subnet.param_dict
    mofa_params = mofa.param_dict
    
    with torch.no_grad():
        mofa.classifier.weight.copy_(subnet.classifier.weight)
        mofa.classifier.bias.copy_(subnet.classifier.bias)
        for u_ind, unit in enumerate(subnet.units):
                for l_ind, layer in enumerate(unit.layers):
                    mofa.units[u_ind].layers[l_ind].conv2d.weight.copy_(subnet.units[u_ind].layers[l_ind].conv2d.weight)
                    mofa.units[u_ind].layers[l_ind].ktm.copy_(subnet.units[u_ind].layers[l_ind].ktm)
                    if mofa.bias_list[u_ind][l_ind] is True:
                        mofa.units[u_ind].layers[l_ind].conv2d.bias.copy_(subnet.units[u_ind].layers[l_ind].conv2d.bias)
    return mofa
                

# def sample_subnet_width(mofa):
 #TODO
    
# def update_mofa_from_subnet_width(mofa, subnet):
 #TODO

## Others

In [58]:
def cross_entropy_loss_with_soft_target(pred, soft_target):
    logsoftmax = nn.LogSoftmax()
    return torch.mean(torch.sum(- soft_target * logsoftmax(pred), 1))

# <ins> MOFA Training </ins>

In [7]:
n_units = 5
n_layers = 3

param_dict = {}
param_dict['n_units']     = n_units
param_dict['in_ch']       = 3
param_dict['out_class']   = 100
param_dict['width_list']  = [[256]*n_layers for _ in range(n_units)]
param_dict['kernel_list'] = [[3]*n_layers for _ in range(n_units)]
param_dict['bias_list']   = [[True]*n_layers for _ in range(n_units)]
param_dict['bn']          = True


In [78]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

class Args():
    def __init__(self):
        super(Args, self).__init__()
        self.truncate_testset = False
        self.act_mode_8bit = False
        
args = Args()
train_dataset, test_dataset = cifar100_get_datasets(('data', args))

trainset = DataLoader(dataset=train_dataset,
                      batch_size=100,
                      shuffle=True,
                      num_workers=0)

valset = DataLoader(dataset=test_dataset,
                      batch_size=1000,
                      shuffle=False,
                      num_workers=0)

mofa = MOFAnet(param_dict)
mofa = mofa.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mofa.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=35, gamma=0.4)

best_val_accuracy = 0
max_epochs = 150
for epoch in range(max_epochs):
    t0 = time.time()
    mofa.train()
    for batch, labels in trainset:
        batch, labels = batch.to(device), labels.to(device)
        
        y_pred = mofa(batch)
        loss = criterion(y_pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()
    
    print(f'Epoch {epoch+1}')
    print(f'\tTraining loss:{loss.item()}')
    t1 = time.time()
    print(f'\tTraining time:{t1-t0:.2f} s - {(t1-t0)/60:.2f} mins ')
    # Validation
    correct = 0
    total = 0
    mofa.eval()
    with torch.no_grad():
        for data in valset:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = mofa(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    if val_accuracy > best_val_accuracy:
        if epoch != 0:
            os.remove(f'mofa_models/noclamp_mofa_acc{100*best_val_accuracy:.0f}%.pth.tar')
        torch.save(mofa, f'mofa_models/noclamp_mofa_acc{100*val_accuracy:.0f}%.pth.tar')
        best_val_accuracy = val_accuracy
    print('\tAccuracy of the mofa on the test images: %d %%' % (
        100 * correct / total))
    

Files already downloaded and verified
Files already downloaded and verified
Epoch 1
	Training loss:3.21022367477417
	Training time:28.91 s - 0.48 mins 
	Accuracy of the mofa on the test images: 19 %
Epoch 2
	Training loss:2.6007723808288574
	Training time:29.35 s - 0.49 mins 
	Accuracy of the mofa on the test images: 30 %
Epoch 3
	Training loss:2.3514862060546875
	Training time:29.56 s - 0.49 mins 
	Accuracy of the mofa on the test images: 33 %
Epoch 4
	Training loss:1.8340553045272827
	Training time:29.65 s - 0.49 mins 
	Accuracy of the mofa on the test images: 44 %
Epoch 5
	Training loss:1.9492859840393066
	Training time:29.69 s - 0.49 mins 
	Accuracy of the mofa on the test images: 49 %
Epoch 6
	Training loss:1.6834275722503662
	Training time:29.71 s - 0.50 mins 
	Accuracy of the mofa on the test images: 53 %
Epoch 7
	Training loss:1.5695773363113403
	Training time:29.73 s - 0.50 mins 
	Accuracy of the mofa on the test images: 54 %
Epoch 8
	Training loss:1.3024191856384277
	Training

KeyboardInterrupt: 

# <ins> Elastic Kernel Training </ins>

In [8]:
model = torch.load('mofa_models/noclamp_mofa_acc71%.pth.tar')
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [14]:
fused_model = fuse_bn_mofa(model)
mofa = copy.deepcopy(fused_model)
fused_model = fused_model.to(device)
mofa = mofa.to(device)

In [16]:
kd_ratio = 0.5

class Args():
    def __init__(self):
        super(Args, self).__init__()
        self.truncate_testset = False
        self.act_mode_8bit = False
        
args = Args()
train_dataset, test_dataset = cifar100_get_datasets(('data', args))

trainset = DataLoader(dataset=train_dataset,
                      batch_size=100,
                      shuffle=True,
                      num_workers=0)

valset = DataLoader(dataset=test_dataset,
                      batch_size=1000,
                      shuffle=False,
                      num_workers=0)

criterion = torch.nn.CrossEntropyLoss()

best_val_accuracy = 0
max_epochs = 250
for epoch in range(max_epochs):
    t0 = time.time()
    mofa.train()
    for batch, labels in trainset:
        batch, labels = batch.to(device), labels.to(device)
        
#         mofa = make_bn_stats_false(mofa)
        subnet = sample_subnet_kernel(mofa)
        subnet = subnet.to(device)
        optimizer = torch.optim.SGD(subnet.parameters(), lr=1e-3)
      
        y_pred = subnet(batch)
        
        if kd_ratio > 0:
            fused_model.train()
            with torch.no_grad():
                soft_logits = fused_model(batch).detach()
                soft_label = F.softmax(soft_logits, dim=1)
            kd_loss = cross_entropy_loss_with_soft_target(y_pred, soft_label)
            loss = kd_ratio * kd_loss + criterion(y_pred, labels)
        else:
            loss = criterion(y_pred, labels)     
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        mofa = update_mofa_from_subnet_kernel(mofa, subnet)
        
    print(f'Epoch {epoch+1}')
    print(f'\tTraining loss:{loss.item()}')
    t1 = time.time()
    print(f'\tTraining time:{t1-t0:.2f} s - {(t1-t0)/60:.2f} mins ')
    
    # Validation
    correct = 0
    total = 0
    mofa.eval()
    with torch.no_grad():
#         mofa = make_bn_stats_true(mofa)
#         mofa.train()
#         for data in valset:
#             images, labels = data
#             images, labels = images.to(device), labels.to(device)
#             outputs = mofa(images)
#         mofa.eval()
        for data in valset:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = mofa(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
#     if val_accuracy > best_val_accuracy:
#         if epoch is not 0:
#             os.remove(f'mofa_models/ofa_acc{100*best_val_accuracy:.0f}%.pth.tar')
#         torch.save(mofa, f'mofa_models/ofa_acc{100*val_accuracy:.0f}%.pth.tar')
#         best_val_accuracy = val_accuracy
    print('\tAccuracy of the mofa on the test images: %d %%' % (
        100 * correct / total))
    print(f'\tFirst ktm: {mofa.units[0].layers[0].ktm[4].item()}')
    print(f'\tLast ktm: {mofa.units[4].layers[2].ktm[4].item()}')
    

Files already downloaded and verified
Files already downloaded and verified


<ipython-input-15-9ec76991f31d>:3: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.mean(torch.sum(- soft_target * logsoftmax(pred), 1))


Epoch 1
	Training loss:6.849119663238525
	Training time:58.44 s - 0.97 mins 
	Accuracy of the mofa on the test images: 63 %
	First ktm: 1.0893168449401855
	Last ktm: 1.0
Epoch 2
	Training loss:1.3607887029647827
	Training time:58.56 s - 0.98 mins 
	Accuracy of the mofa on the test images: 65 %
	First ktm: 1.2011218070983887
	Last ktm: 1.0
Epoch 3
	Training loss:0.793714165687561
	Training time:58.43 s - 0.97 mins 
	Accuracy of the mofa on the test images: 66 %
	First ktm: 1.2770159244537354
	Last ktm: 1.0
Epoch 4
	Training loss:1.04897141456604
	Training time:58.57 s - 0.98 mins 
	Accuracy of the mofa on the test images: 66 %
	First ktm: 1.3380467891693115
	Last ktm: 1.0
Epoch 5
	Training loss:1.8602148294448853
	Training time:58.25 s - 0.97 mins 
	Accuracy of the mofa on the test images: 66 %
	First ktm: 1.3763775825500488
	Last ktm: 1.0
Epoch 6
	Training loss:1.184467077255249
	Training time:58.26 s - 0.97 mins 
	Accuracy of the mofa on the test images: 66 %
	First ktm: 1.41500926017

KeyboardInterrupt: 

# <ins> Testing <ins/>

**MOFA Test**

In [97]:
correct = 0
total = 0
mofa.eval()
with torch.no_grad():
    for data in valset:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = mofa(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(correct / total)

0.6512


**Random Subnet Test**

In [44]:
subnet = sample_subnet_kernel(mofa)

In [45]:
print(subnet.param_dict)

{'n_units': 5,
 'in_ch': 3,
 'out_class': 100,
 'width_list': [[256, 256, 256],
  [256, 256, 256],
  [256, 256, 256],
  [256, 256, 256],
  [256, 256, 256]],
 'kernel_list': [[1, 3, 1], [3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3]],
 'bias_list': [[True, True, True],
  [True, True, True],
  [True, True, True],
  [True, True, True],
  [True, True, True]],
 'bn': False}

In [46]:
correct = 0
total = 0
subnet = subnet.to(device)
subnet.eval()
with torch.no_grad():
    for data in valset:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = subnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        break
#     val_accuracy = correct / total
#     if val_accuracy > best_val_accuracy:
#         if epoch is not 0:
#             os.remove(f'mofa_models/ofa_acc{100*best_val_accuracy:.0f}%.pth.tar')
#         torch.save(mofa, f'mofa_models/ofa_acc{100*val_accuracy:.0f}%.pth.tar')
#         best_val_accuracy = val_accuracy
print(correct / total)

0.674
